# All value we need
# Parameters
worda_count (int) – Number of occurrences for first word.
wordb_count (int) – Number of occurrences for second word.
bigram_count (int) – Number of co-occurrences for phrase “worda_wordb”.
len_vocab (int) – Not used.
min_count (int) – Ignore all bigrams with total collected count lower than this value.
corpus_word_count (int) – Total number of words in the corpus.
Returns
If bigram_count >= min_count, return the collocation score, in the range -1 to 1. Otherwise return -inf.

Return type
float

In [12]:
import typing
from typing import  List, Any, Dict, Tuple
from itertools import combinations


import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import nltk
from nltk.corpus import stopwords



import importlib
import capturePPMI
from capturePPMI import Corpus
importlib.reload(capturePPMI)
from capturePPMI import capture_cosine_similarity , normalize_and_find_mean , get_sorted_positive_words




#  A half self defined NPPMI calculate class

In [13]:
# corpus_path = r"/Users/chenyujie/Desktop/ClassifiedData/Reddit/2014_02_done.csv"
# df = pd.read_csv(corpus_path, header='infer',)
# corpus = df['text'].tolist()
# test_corpus = Corpus(corpus)
# test_corpus.counting_occur()
# test_corpus.calculate_npmi_vectors()

In [14]:
def get_nppmi_matrix (file_path: str):
    df = pd.read_csv(file_path, header='infer')
    df = df.dropna(subset=['text'])
    corpus = df['text'].tolist()
    test_corpus = Corpus(corpus)
    test_corpus.counting_occur()
    test_corpus.calculate_npmi_vectors()
    return test_corpus.npmi_vectors

In [15]:
# file_paths = ["/Users/chenyujie/Desktop/ClassifiedData/Reddit/RC_2009-04_done.csv",
#              "/Users/chenyujie/Desktop/ClassifiedData/Reddit/RC_2010-01_done.csv",
#              "/Users/chenyujie/Desktop/ClassifiedData/Reddit/RC_2011-02_done.csv",
#              "/Users/chenyujie/Desktop/ClassifiedData/Reddit/2014_02_done.csv"]


file_paths = ["/Users/chenyujie/Desktop/ClassifiedData/Reddit/2007.csv",
              "/Users/chenyujie/Desktop/ClassifiedData/Reddit/2008.csv",
              "/Users/chenyujie/Desktop/ClassifiedData/Reddit/RC_2009-04_done.csv",
             "/Users/chenyujie/Desktop/ClassifiedData/Reddit/RC_2010-01_done.csv",
             "/Users/chenyujie/Desktop/ClassifiedData/Reddit/RC_2011-02_done.csv",
             "/Users/chenyujie/Desktop/ClassifiedData/Reddit/2014_02_done.csv"]

In [16]:
nppmi_martix = [get_nppmi_matrix(file_path) for file_path in file_paths]

Calculating NPMI: 100%|██████████| 1351115/1351115 [00:00<00:00, 11198162.78it/s]


In [17]:
slut_words = ["nigger","ass","motherfucker","faggot"] # Using word 'ass' instead of used word 'asshole', the word 'ass' occur more frequent


In [18]:
test_vectors = [  nppmi_martix[0][word]  for word in slut_words ]

In [19]:
test_vectors_mean = normalize_and_find_mean(test_vectors)

In [20]:
print (test_vectors_mean)

[-8.69061942e-06  4.58704541e-05 -3.35321543e-05 ...  0.00000000e+00
 -5.68828316e-02  0.00000000e+00]


In [21]:
def sim_top(vectors: Dict[str, np.ndarray], word: np.ndarray, num_words: int = 10) -> List[Tuple[str, float]]:
    """
    Return the top-k words with the highest cosine similarity to a given word vector.

    Args:
    vectors (Dict[str, np.ndarray]): Dictionary of word vectors.
    word (np.ndarray): The word vector for which to find similar words.
    num_words (int, optional): The number of top similar words to return. Default is 10.

    Returns:
    List[Tuple[str, float]]: List of tuples containing the word and its cosine similarity to the given word vector.
    """
    # Function to calculate cosine similarity
    def cosine_similarity(vec1: np.ndarray, vec2: np.ndarray) -> float:
        norm1 = np.linalg.norm(vec1)
        norm2 = np.linalg.norm(vec2)
        if norm1 == 0 or norm2 == 0:
            return 0.0  # Avoid division by zero
        similarity = np.dot(vec1, vec2) / (norm1 * norm2)
        return similarity

    # Calculate similarity with each word in the dictionary
    similarities = {}
    for key, vector in vectors.items():
        sim = cosine_similarity(vector, word)
        similarities[key] = sim

    # Sort the dictionary by similarity in descending order
    sorted_similarities = sorted(similarities.items(), key=lambda item: item[1], reverse=True)

    # Return the top-k words and their similarities
    return sorted_similarities[:num_words]

# Example usage:
# Assuming 'vectors_dict' is your dictionary of word vectors and 'query_vector' is the vector you want to compare
# vectors_dict = {
#     'apple': np.array([1, 2, 3]),
#     'banana': np.array([4, 5, 6]),
#     'cherry': np.array([7, 8, 9]),
#     ...
# }
# query_vector = np.array([1, 2, 3])
top_similar_words = sim_top(nppmi_martix[0], test_vectors_mean, num_words=100)
print(top_similar_words)


[('ass', 0.6204799372057819), ('nigger', 0.6092721406432279), ('faggot', 0.5972104963068908), ('motherfucker', 0.5553181565890716), ('I', 0.5009209897662708), ('that', 0.48771930204906255), ('it', 0.48476631233774936), ('not', 0.4839465735371006), ('for', 0.4837831786950401), ('in', 0.48231313171742207), ('they', 0.4808503149976603), ('be', 0.48029868338764925), ('all', 0.47882978654491154), ('he', 0.47403460139036996), ('have', 0.4730881582405251), ('you', 0.4714816454761781), ('do', 0.4689025104271288), ('with', 0.46290244986874884), ('of', 0.46099153595676423), ('and', 0.4603766326940414), ('on', 0.4591033478331324), ('your', 0.4555732809356733), ('will', 0.4550417166522111), ('as', 0.45456496125554746), ('would', 0.4527804425174777), ('we', 0.4522697703733835), ('like', 0.4514796086452129), ('but', 0.449839828218796), ('get', 0.448009883078174), ('a', 0.4473301993211366), ('or', 0.4473111201646439), ('can', 0.4472459247497004), ('go', 0.4447904291249709), ('so', 0.44370071025453506

# If we remove all stop of,this show certain level of sense, but no good as the word2vec method

In [22]:
# load the word
shifting_words = get_sorted_positive_words('/Users/chenyujie/Library/CloudStorage/OneDrive-QueenMary,UniversityofLondon/tensorflow-test/coefs_with_fns_2023.pkl')

In [23]:
shifting_words_vector = {}
for word in shifting_words:
    vectors = [None] * len(nppmi_martix) # Initialize a list to hold vectors from each matrix
    skip_word = False  # Flag to determine if word should be skipped

    # Iterate over each matrix in nppmi_matrix
    for i, matrix in enumerate(nppmi_martix):
        if word in matrix:
            vectors[i] = matrix[word]
        else:
            skip_word = True  # Skip this word if it's not present in any of the matrices
            break  # Exit the inner loop early if word is not found in the current matrix

    # Only add the word and its vectors to the dictionary if the word wasn't skipped
    if not skip_word:
        shifting_words_vector[word] = vectors

# Example usage
# Here `shifting_words_vector` will contain words as keys and a list of vectors as values, only for those words found in all matrices


In [24]:
shifting_words_vector

{'by': [array([-5.85507749e-04, -7.52210538e-04,  1.40632702e-02, ...,
         -8.81716119e-01, -8.81716119e-01, -8.52325581e-01]),
  array([-3.58384488e-05, -4.46565779e-05,  7.84460866e-03, ...,
          0.00000000e+00, -3.74494428e-01,  0.00000000e+00]),
  array([-5.82602877e-04, -7.57556085e-04,  1.57522013e-02, ...,
          0.00000000e+00, -9.30455635e-01,  0.00000000e+00]),
  array([-2.13229367e-04, -2.88151410e-04, -4.06927363e-04, ...,
         -7.25046898e-01, -7.50048515e-01,  0.00000000e+00]),
  array([-9.17424784e-05, -1.29764512e-04, -1.69785387e-04, ...,
         -5.68878391e-01, -6.09512562e-01, -5.89195476e-01]),
  array([-1.58322496e-04, -2.31273042e-04, -2.80085962e-04, ...,
          0.00000000e+00, -7.17623804e-01,  0.00000000e+00])],
 'hypocritical': [array([-0.01970187, -0.04466698, -0.06978471, ...,  0.        ,
          0.        ,  0.        ]),
  array([ 0.00356253, -0.00332932, -0.00525468, ...,  0.        ,
          0.        ,  0.        ]),
  array([

In [25]:
from capturePPMI import capture_cosine_similarity

def find_sim(words_vectors: Dict[str, List[np.ndarray]], words: List[str], centre_word_vector: List[np.ndarray]) -> Dict[str, List[float]]:
    """
    Find the similarity of vectors of words to corresponding center word vectors.

    Args:
    words_vectors (Dict[str, List[np.ndarray]]): Dictionary mapping words to lists of their vectors.
    words (List[str]): List of words to find similarities for.
    centre_word_vector (List[np.ndarray]): List of central word vectors, one for each word.

    Returns:
    Dict[str, List[float]]: Dictionary mapping words to their similarity scores with the center word vectors.
    """
    similarity_scores = {}
    for word in words:
        if word in words_vectors:
            vectors = words_vectors[word]
            word_similarity = [capture_cosine_similarity(vector, centre_word_vector[i]) for i, vector in enumerate(vectors)]
            similarity_scores[word] = word_similarity
        # else:
        #     # Handling cases where the word might not be in the dictionary
        #     similarity_scores[word] = f'Word {word} not in dictionary'

    return similarity_scores

In [26]:
def calculate_similarity_gradient(similarities: List[float]) -> List[float]:
    """
    Calculate the gradient of similarity scores between consecutive elements.

    Args:
    similarities (List[float]): List of similarity scores for a word across different models or times.

    Returns:
    List[float]: List of gradients between consecutive similarity scores.
    """
    # Calculate the gradients between consecutive similarity scores
    gradients = [similarities[i] - similarities[i - 1] for i in range(1, len(similarities))]
    return gradients

In [39]:
def sort_words_by_sum_gradient(gradient_data: Dict[str, List[float]], sim: Dict[str, List[float]], min_threshold: float = None, max_threshold: float = None) -> List[str]:
    """
    Sort words based on the sum of their gradient values, excluding words that do not meet the specified similarity thresholds.

    Args:
    gradient_data (Dict[str, List[float]]): Dictionary of words and their list of gradient values.
    sim (Dict[str, List[float]]): Dictionary of words and their list of similarity scores.
    min_threshold (float, optional): Minimum similarity score threshold to include a word in the sorting.
    max_threshold (float, optional): Maximum similarity score threshold to include a word in the sorting.

    Returns:
    List[str]: List of words sorted by the sum of their gradient values, descending, that meet the similarity score thresholds.
    """
    # Filter to ignore empty lists, non-list items, and filter by similarity thresholds
    filtered_data = {word: gradients for word, gradients in gradient_data.items()
                     if isinstance(gradients, list) and gradients and
                        (min_threshold is None or max(sim.get(word, [0])) >= min_threshold) and
                        (max_threshold is None or max(sim.get(word, [0])) <= max_threshold)}

    # Sort words based on the sum of the gradient values
    sorted_words = sorted(filtered_data, key=lambda x: sum(filtered_data[x]), reverse=True)
    return sorted_words

In [40]:
mean_vectors = [None] * len(nppmi_martix)
for i in range (len (mean_vectors)):
    mean_vectors[i] = normalize_and_find_mean([  nppmi_martix[i][word]  for word in slut_words])


In [41]:
shifting_words_vector_sim = find_sim (shifting_words_vector, shifting_words,mean_vectors )

In [42]:
print (  shifting_words_vector_sim['ass'][1])

0.6518359329863807


In [54]:
gradient_results = {}
for word, scores in shifting_words_vector_sim.items():
    gradient_results[word] = calculate_similarity_gradient(scores)

In [55]:
# increasing_words, decreasing_words = categorize_gradient_words(gradient_results)


In [56]:
for word in slut_words:
    print ("word",word ,gradient_results[word])
    print (shifting_words_vector_sim[word])

word nigger [-0.00019185465943605262, -0.044907080877825356, 0.06923700363575092, -0.042202197912063255, 0.00045796974812717384]
[0.6092721406432279, 0.6090802859837918, 0.5641732051059665, 0.6334102087417174, 0.5912080108296541, 0.5916659805777813]
word ass [0.03135599578059878, 0.011815647628154236, -0.013351865415790365, 0.01699696890850444, -0.014151199312668283]
[0.6204799372057819, 0.6518359329863807, 0.6636515806145349, 0.6502997151987445, 0.667296684107249, 0.6531454847945807]
word motherfucker [0.018932540992790847, 0.020640614973777005, -0.020451790089049293, 0.01541895688522077, -0.0070847710116686]
[0.5553181565890716, 0.5742506975818624, 0.5948913125556394, 0.5744395224665901, 0.5898584793518109, 0.5827737083401423]
word faggot [-0.0005704407755682261, -0.001690853291116734, -0.001761135004678449, 0.036849609553057716, 0.005243013033814359]
[0.5972104963068908, 0.5966400555313226, 0.5949492022402059, 0.5931880672355274, 0.6300376767885851, 0.6352806898223995]


In [59]:
min_threshold_value = -1 # Define a minimum threshold for maximum similarity
max_threshold_value = 0.5 # 0.55 # Define a maximum threshold for maximum similarity


words_sorted_by_sum = sort_words_by_sum_gradient(gradient_results, shifting_words_vector_sim, min_threshold=min_threshold_value, max_threshold=max_threshold_value)

print (len(words_sorted_by_sum))


for word in words_sorted_by_sum:
    print ("word",word,gradient_results[word])
    print (shifting_words_vector_sim[word])


701
word bitch [0.09758994306056534, 0.005035791885428353, 0.009119123719652789, 0.07204297477161076, -0.024946525483666937]
[0.2773056377613189, 0.37489558082188423, 0.3799313727073126, 0.3890504964269654, 0.46109347119857613, 0.4361469457149092]
word girl [0.05863183434858005, 0.06820957562625818, 0.025160061493281816, 0.03378384764801917, -0.028238030302090156]
[0.26125885347137645, 0.3198906878199565, 0.3881002634462147, 0.4132603249394965, 0.44704417258751566, 0.4188061422854255]
word dick [0.03719872866843807, 0.0640169868425346, -0.030495422032522423, 0.04794352619364939, 0.029060740841437505]
[0.23101015844155423, 0.2682088871099923, 0.3322258739525269, 0.30173045192000447, 0.34967397811365386, 0.37873471895509137]
word lol [0.05851143590871721, 0.017662070987267076, -0.04337688655577415, 0.08374824579349052, 0.029175391232827697]
[0.15267179303151115, 0.21118322894022837, 0.22884529992749544, 0.1854684133717213, 0.2692166591652118, 0.2983920503980395]
word ya [0.06520013563075

In [60]:
def categorize_words_by_gradient_sign( words:List[str] ,gradient_data: Dict[str, List[float]] ) -> (List[str], List[str]):
    """
    Categorize words into two lists based on their gradient signs: all positive or all negative.

    Args:
    gradient_data (Dict[str, List[float]]): Dictionary of words and their list of gradient values.

    Returns:
    Tuple[List[str], List[str]]:
        - First list contains words that have all positive gradient values.
        - Second list contains words that have all negative gradient values.
    """
    all_positive_words = []
    all_negative_words = []

    for word, gradients in gradient_data.items():
        if word not in words:
            continue
        if all(g > 0 for g in gradients):
            all_positive_words.append(word)
        elif all(g < 0 for g in gradients):
            all_negative_words.append(word)

    return all_positive_words, all_negative_words


all_positive, all_negative = categorize_words_by_gradient_sign(words_sorted_by_sum , gradient_results)
print("Words with all positive gradients:", len(all_positive))
for word in all_positive:
    print (word, "\n", gradient_results[word],"\n",shifting_words_vector_sim[word])
print("\nWords with all negative gradients:",len(all_negative))
for word in all_negative:

    print (word, "\n", gradient_results[word],"\n",shifting_words_vector_sim[word])


Words with all positive gradients: 6
start 
 [0.026623258882228273, 0.007745155761669564, 0.010573428432172904, 0.005206895648104559, 0.011388706738736654] 
 [0.2665435375858569, 0.2931667964680852, 0.30091195222975475, 0.31148538066192766, 0.3166922763100322, 0.32808098304876887]
grab 
 [0.007105614224668821, 0.015683845939398713, 0.011460288032199839, 0.015472970896975025, 0.012565139075088133] 
 [0.09530765738041884, 0.10241327160508766, 0.11809711754448637, 0.1295574055766862, 0.14503037647366124, 0.15759551554874937]
horrible 
 [0.03162457705646117, 0.0021699503351840926, 0.007308070851504853, 0.01121884172455509, 0.003338820598032949] 
 [0.08838531148015803, 0.1200098885366192, 0.1221798388718033, 0.12948790972330815, 0.14070675144786324, 0.1440455720458962]
yo 
 [0.036772294780774525, 0.001914259660829587, 0.015221037295664763, 0.009308696226059288, 0.0683326117659791] 
 [0.09752740328875621, 0.13429969806953074, 0.13621395773036032, 0.1514349950260251, 0.16074369125208438, 0.22

In [63]:
from prettytable import PrettyTable

# Function to print results in table format
def print_results(title, words, gradient_results, shifting_words_vector_sim):
    print(title)
    table = PrettyTable()
    table.field_names = ["Word","Similarity Scores"]

    for word in words:
        gradients = gradient_results[word]
        similarities = shifting_words_vector_sim[word]
        table.add_row([word, similarities])

    print(table)

# Example usage:
# Assuming all_positive, all_negative, gradient_results, and shifting_words_vector_sim are defined

print_results("Words with  positive gradients:", all_positive, gradient_results, shifting_words_vector_sim)
print("\nWords with all negative gradients:", len(all_negative))
print_results("Words with  negative gradients:", all_negative, gradient_results, shifting_words_vector_sim)


Words with  positive gradients:
+----------+--------------------------------------------------------------------------------------------------------------------------------+
|   Word   |                                                       Similarity Scores                                                        |
+----------+--------------------------------------------------------------------------------------------------------------------------------+
|  start   |  [0.2665435375858569, 0.2931667964680852, 0.30091195222975475, 0.31148538066192766, 0.3166922763100322, 0.32808098304876887]   |
|   grab   | [0.09530765738041884, 0.10241327160508766, 0.11809711754448637, 0.1295574055766862, 0.14503037647366124, 0.15759551554874937]  |
| horrible |  [0.08838531148015803, 0.1200098885366192, 0.1221798388718033, 0.12948790972330815, 0.14070675144786324, 0.1440455720458962]   |
|    yo    | [0.09752740328875621, 0.13429969806953074, 0.13621395773036032, 0.1514349950260251, 0.16074369125208438

In [61]:
all_positive, all_negative


(['start', 'grab', 'horrible', 'yo', 'park', 'pant'],
 ['bush', 'americans', 'sheeple', 'fact'])

# Result from other model

In [72]:
word2vec_list = ['laden','man']
print_results("In PPMI embedding with words from word2vec", word2vec_list, gradient_results, shifting_words_vector_sim)


In PPMI embedding with words from word2vec
+-------+-------------------------------------------------------------------------------------------------------------------------------+
|  Word |                                                       Similarity Scores                                                       |
+-------+-------------------------------------------------------------------------------------------------------------------------------+
| laden | [0.1067549224672562, 0.11401801862872152, 0.10361676608760263, 0.08543137236701119, 0.06622197184602513, 0.04858344581324231] |
|  man  |  [0.36540382914014946, 0.38760649069145575, 0.4191126746369291, 0.4031037009441744, 0.44855774776810303, 0.4274802395948407]  |
+-------+-------------------------------------------------------------------------------------------------------------------------------+


In [73]:
GloVe_word = ['bush', 'throat', 'barely', 'hate']
print_results("In PPMI embedding with words from  GloVE", GloVe_word, gradient_results, shifting_words_vector_sim)


In PPMI embedding with words from  GloVE
+--------+--------------------------------------------------------------------------------------------------------------------------------+
|  Word  |                                                       Similarity Scores                                                        |
+--------+--------------------------------------------------------------------------------------------------------------------------------+
|  bush  |  [0.295442796889861, 0.25975417213044344, 0.1888394850937837, 0.15597502527938892, 0.13015463641877617, 0.10561900752677282]   |
| throat |  [0.1342506512519112, 0.11116620293413502, 0.16256774834743817, 0.10614446309963123, 0.1319260466446401, 0.10390863813431252]  |
| barely | [0.050607875515308244, 0.08836847667481798, 0.0643132038712841, 0.10574804863025322, 0.10658283388989503, 0.12341784985414093] |
|  hate  |   [0.29740794264891285, 0.33012175706168617, 0.3785999185740081, 0.31600514585503237, 0.3802718438677825, 0.

# All words

In [67]:
def print_results_withGrid(title, words, gradient_results, shifting_words_vector_sim):
    print(title)
    table = PrettyTable()
    table.field_names = ["Word","Similarity Scores","gradient"]

    for word in words:
        if gradient_results.get(word, None) is None:
            continue
        gradients = gradient_results[word]
        similarities = shifting_words_vector_sim[word]
        table.add_row([word, similarities,gradients])

    print(table)
print_results_withGrid("All words in PPMI", shifting_words, gradient_results, shifting_words_vector_sim)


All words in Word2vec
+---------------+----------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------+
|      Word     |                                                        Similarity Scores                                                         |                                                        gradient                                                        |
+---------------+----------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------+
|       by      |    [0.4188366034253412, 0.4369954897399056, 0.4649614954659315, 0.45128740912579424, 0.4726584205563148, 0.4494413141821449]     |    [0.0181588863145